In [944]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-aug-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2022/train.csv
/kaggle/input/tabular-playground-series-aug-2022/test.csv


In [945]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-aug-2022/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-aug-2022/test.csv")

In [946]:
train=train.drop('id', axis=1)
train=train.drop('product_code', axis=1)
train=train.drop('attribute_0', axis=1)
train=train.drop('attribute_1', axis=1)
train=train.drop('attribute_2', axis=1)
train=train.drop('attribute_3', axis=1)
train =train.fillna(train.mean())

In [947]:
train.head()

,loading,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,80.10,7,8,4,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594000,15.193,15.029,16.048444,13.034,14.684,764.100,0
1,84.89,14,3,3,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915000,11.755,14.732,15.425000,14.395,15.631,682.057,0
2,82.43,12,1,5,18.057,11.652,16.738,18.240,12.718,18.288,12.715,15.607,19.172085,13.798,16.711,18.631000,14.094,17.946,663.376,0
3,101.07,13,2,6,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377000,10.020,15.250,15.562000,16.154,17.172,826.282,0
4,188.06,9,2,8,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932000,12.428,16.182,12.760000,13.153,16.412,579.885,0


In [948]:
X = train.drop("failure", axis=1)
y = train["failure"]

In [949]:
from sklearn.model_selection import train_test_split


In [950]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.001,random_state =50)

# 1.ElasticNet


In [951]:
from sklearn.linear_model import ElasticNet
import warnings; 
warnings.simplefilter('ignore')

elasticNet=ElasticNet(alpha=0.1,l1_ratio=0.6)
elasticNet.fit(X_train,y_train)

ElasticNet(alpha=0.1, l1_ratio=0.6)

# 2.ElasticNetCV

In [952]:
from sklearn.linear_model import ElasticNetCV
alphas = [0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1]
l1_ratio=[0.01,0.2,0.4,0.6,0.8,0.9,0.91,0.92,0.93,0.94,1]
elasticNetCV=ElasticNetCV(alphas=alphas,l1_ratio=l1_ratio, cv=5)
elasticNetCV.fit(X_train,y_train)


ElasticNetCV(alphas=[0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1], cv=5,
             l1_ratio=[0.01, 0.2, 0.4, 0.6, 0.8, 0.9, 0.91, 0.92, 0.93, 0.94,
                       1])

# 3.AdaBoostClassifier

In [953]:
from sklearn.ensemble import AdaBoostClassifier
adaBoostClassifier = AdaBoostClassifier(n_estimators=100,random_state=100)
adaBoostClassifier.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100, random_state=100)

# 4.XGBClassifier

In [954]:
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
model = XGBClassifier(
    n_estimators=30,
    learning_rate =0.3,
    max_depth=3,
    min_child_weight=1,
    gamma=0.3,
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=12,
    scale_pos_weight=1,
    reg_lambda=1,
    seed=27)


xGBClassifier=model.fit(X_train, y_train)

# 5.logisticRegression

In [955]:
from sklearn.linear_model import LogisticRegression
logisticRegression = LogisticRegression(**{'penalty': 'elasticnet', 'tol': 0.01167, 'C': 0.3, 'fit_intercept': True, 'solver': 'saga', 'max_iter': 3000, 'l1_ratio':0.5})

logisticRegression.fit(X_train,y_train)

LogisticRegression(C=0.3, l1_ratio=0.5, max_iter=3000, penalty='elasticnet',
                   solver='saga', tol=0.01167)

# 6.lightgbm

In [956]:
import lightgbm as lgb
# configurations
lgbm_params = {
    "metric":'binary_logloss',
    "objective":"binary",
    'learning_rate': 0.01,
    'feature_fraction': 0.90, 
    'bagging_fraction': 0.90,
    'boosting': 'gbdt'
}
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

lightgbm_model = lgb.train(lgbm_params,
                           train_set=lgb_train,
                           num_boost_round= 1200,
                           valid_sets=lgb_eval,
                           early_stopping_rounds=600)

[LightGBM] [Info] Number of positive: 5646, number of negative: 20897
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006186 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4161
[LightGBM] [Info] Number of data points in the train set: 26543, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.212711 -> initscore=-1.308658
[LightGBM] [Info] Start training from score -1.308658
[1]	valid_0's binary_logloss: 0.384293
Training until validation scores don't improve for 600 rounds
[2]	valid_0's binary_logloss: 0.38404
[3]	valid_0's binary_logloss: 0.383814
[4]	valid_0's binary_logloss: 0.383581
[5]	valid_0's binary_logloss: 0.383345
[6]	valid_0's binary_logloss: 0.383209
[7]	valid_0's binary_logloss: 0.382961
[8]	valid_0's binary_logloss: 0.382911
[9]	valid_0's binary_logloss: 0.382961
[10]	valid_0's binary_logloss: 0.382889
[11]	valid_0's binary_logloss: 0.382806
[12]	valid_0

# 7.Save Models

In [957]:
import pickle
filename = "elasticNetCV.pickle"
pickle.dump(elasticNetCV, open(filename, "wb"))
filename = "XGBClassifier.pickle"
pickle.dump(xGBClassifier, open(filename, "wb"))
filename = "AdaBoostClassifier.pickle"
pickle.dump(adaBoostClassifier, open(filename, "wb"))
filename = "elasticNet.pickle"
pickle.dump(elasticNet, open(filename, "wb"))
filename = "logisticRegression.pickle"
pickle.dump(logisticRegression, open(filename, "wb"))
lightgbm_model.save_model('model.txt')